<a href="https://colab.research.google.com/github/yongchanzzz/FSEC2csv/blob/main/FSEC2csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FSEC to csv converter
FSEC2csv merges ASCII files from a Shimadzu HPLC and writes out a single csv file for later use.


In [ ]:
#@title Import necessary libraries and upload files
import re
import csv
from google.colab import files
from datetime import datetime

uploaded_files = files.upload()

In [ ]:
#@title Choose a channel
channel = "B-Ch1" #@param ["B-Ch1", "B-Ch2", "A-Ch1"]

In [ ]:
#@title Run codes and generate csv

def process_file_content(file_content, channel):
    # Process file content and extract time and data values
    start_pattern = re.compile(r"\[LC Chromatogram\(Detector " + re.escape(channel) + r"\)\]")
    end_pattern = re.compile(r"^\s*$")
    data_values = []
    time_values = []
    time_header = "time"  # Default header for time
    data_headers = []

    # Try different encodings to read the file
    for encoding in ["utf-8", "ISO-8859-1", "windows-1252"]:
        try:
            content = file_content.decode(encoding).split("\n")
            break
        except UnicodeDecodeError:
            continue
    else:
        raise UnicodeDecodeError("Could not decode the file with any of the provided encodings.")

    header_processed = False
    inside_section = False
    for line in content:
        if not header_processed and "Sample Name" in line:
            # Extract headers
            parts = line.strip().split("\t")
            time_header = parts[0].replace("Sample Name", "time")
            data_headers = parts[1:]  # This will be a list of headers after "Sample Name"
            header_processed = True
            continue
        if start_pattern.search(line):
            inside_section = True
            continue
        if inside_section and end_pattern.search(line):
            break
        if inside_section:
            parts = line.strip().split("\t")
            if len(parts) >= 2 and parts[0].replace('.', '', 1).isdigit():
                time_values.append(parts[0])
                data_values.append(parts[1:])
    return time_values, data_values, time_header, data_headers

def process_files(files, channel):
    combined_data = {}
    headers = ["time"]  # Default header for time
    first_file = True

    for filename, file_content in files.items():
        time_values, data_values, time_header, file_data_headers = process_file_content(file_content, channel)
        if first_file:
            headers = [time_header] + file_data_headers  # Update headers from the first file
            first_file = False
        else:
            # Append only data headers (excluding time) from subsequent files
            headers.extend(file_data_headers)

        for time, data in zip(time_values, data_values):
            if time in combined_data:
                combined_data[time].extend(data)
            else:
                # Initialize with empty data for previous files and add data for the current file
                combined_data[time] = [''] * (len(headers) - len(data) - 1) + data

    # Convert the dictionary to a 2D list for CSV writing
    # No sorting applied, preserving the original order
    csv_data = [[time] + combined_data[time] for time in combined_data.keys()]
    return headers, csv_data

def write_csv(headers, data, filename):
    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(headers)  # Write headers to the CSV file
        csvwriter.writerows(data)


In [ ]:
#@title Automatic download
# Generate file name from the current date and time
now = datetime.now()
timestamp_str = now.strftime('%Y%m%d_%H%M%S')
csv_filename = f"FSEC_{timestamp_str}_{channel}.csv"

# Process and write data
headers, processed_data = process_files(uploaded_files, channel)
write_csv(headers, processed_data, csv_filename)
print(f"Data written to {csv_filename}")

files.download(csv_filename)